# Explain Python Model Smoke Tests

This notebook validates that `ModelEngine` can:
- Load model definitions from `definitions/*.json`
- Instantiate all configured models
- Run one simulation step without errors

In [5]:
from pathlib import Path
from model_engine import ModelEngine

repo_root = Path.cwd()
definitions_dir = repo_root / "definitions"

print("Repo root:", repo_root)
print("Definitions dir exists:", definitions_dir.exists())

Repo root: /Users/timantonius/Projects/explain/explain-python
Definitions dir exists: True


In [6]:
definition_files = [
    "baseline.json",
    "aop.json",
    "aph.json",
    "mas.json",
    "pda.json",
    "ventilator.json",
    "resuscitation.json",
    "monitor.json",
    "example_minimal.json",
]

single_results = {}
for file_name in definition_files:
    engine = ModelEngine().load_json_file(definitions_dir / file_name)
    engine.step_model()
    single_results[file_name] = {
        "initialized": engine.is_initialized,
        "model_count": len(engine.models),
        "model_names": sorted(engine.models.keys()),
        "stepsize": engine.modeling_stepsize,
    }

single_results

{'baseline.json': {'initialized': True,
  'model_count': 23,
  'model_names': ['AA',
   'AAR_DA',
   'AA_MVU_ART',
   'DA',
   'DA_PA',
   'FO',
   'LA',
   'LV',
   'MOB',
   'MVU',
   'MVU_ART',
   'MVU_ART_MVU_CAP',
   'MVU_CAP',
   'MVU_CAP_MVU_VEN',
   'MVU_VEN',
   'PA',
   'PDA',
   'RA',
   'RV',
   'SHUNTS',
   'SYS_RES',
   'VEN',
   'VSD'],
  'stepsize': 0.0005},
 'aop.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'aph.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'mas.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'pda.json': {'initialized': True,
  'model_count': 8,
  'model_names': ['AA',
   'AAR_DA',
   'DA',
   'DA_PA',
   'PA',
   'PDA',
   'SYS_RES',
   'VEN'],
  'stepsize': 0.0005},
 'ventilator.json': {'initialized': True,
  'model_count': 10,
  'model_names': 

In [7]:
failed = [
    name
    for name, result in single_results.items()
    if (not result["initialized"]) or result["model_count"] == 0
]

if failed:
    raise AssertionError(f"Smoke test failed for: {failed}")

print("All model definitions loaded and stepped successfully.")

All model definitions loaded and stepped successfully.


In [8]:
from model_engine import ModelEngine
from pathlib import Path

repo_root = Path.cwd()

def run_snapshot(definition_name: str):
    engine = ModelEngine().load_json_file(str(repo_root / "definitions" / definition_name))
    engine.step_model()

    row = {
        "definition": definition_name,
        "has_shunts": "SHUNTS" in engine.models,
        "has_placenta": "PLACENTA" in engine.models,
        "has_fluids": "FL" in engine.models,
        "has_ans": "ANS_AFF" in engine.models and "ANS_EFF" in engine.models,
        "has_ecls": "ECLS" in engine.models,
        "has_ventilator": "VENT" in engine.models,
        "has_resuscitation": "RESUS" in engine.models,
        "has_monitor": "MON" in engine.models,
    }

    shunts = engine.models.get("SHUNTS")
    if shunts is not None:
        row.update({
            "res_fo": round(float(getattr(shunts, "res_fo", 0.0) or 0.0), 3),
            "res_vsd": round(float(getattr(shunts, "res_vsd", 0.0) or 0.0), 3),
            "velocity_fo": round(float(getattr(shunts, "velocity_fo", 0.0) or 0.0), 5),
            "velocity_vsd": round(float(getattr(shunts, "velocity_vsd", 0.0) or 0.0), 5),
        })

    placenta = engine.models.get("PLACENTA")
    if placenta is not None:
        row.update({
            "umb_art_flow": round(float(getattr(placenta, "umb_art_flow", 0.0) or 0.0), 6),
            "umb_ven_flow": round(float(getattr(placenta, "umb_ven_flow", 0.0) or 0.0), 6),
            "placenta_running": bool(getattr(placenta, "placenta_running", False)),
            "umb_clamped": bool(getattr(placenta, "umb_clamped", True)),
        })

    fluids = engine.models.get("FL")
    fluid_target = engine.models.get("VLB")
    if fluids is not None and fluid_target is not None:
        vol_before = float(getattr(fluid_target, "vol", 0.0) or 0.0)
        fluids.add_volume(30, in_time=0.03, fluid_in="normal_saline", site="VLB")
        engine.step_model()
        engine.step_model()
        vol_after = float(getattr(fluid_target, "vol", 0.0) or 0.0)

        row.update({
            "vlb_vol_before": round(vol_before, 6),
            "vlb_vol_after": round(vol_after, 6),
            "fluids_infused": vol_after > vol_before,
        })

    ans_eff = engine.models.get("ANS_EFF")
    ans_target = engine.models.get("R1")
    if ans_eff is not None and ans_target is not None:
        factor_before = float(getattr(ans_target, "r_factor_ps", 1.0) or 1.0)
        engine.step_model()
        factor_after = float(getattr(ans_target, "r_factor_ps", 1.0) or 1.0)

        row.update({
            "ans_factor_before": round(factor_before, 6),
            "ans_factor_after": round(factor_after, 6),
            "ans_effect_applied": factor_after != factor_before,
            "ans_firing_rate": round(float(getattr(ans_eff, "firing_rate", 0.0) or 0.0), 6),
        })

    ecls = engine.models.get("ECLS")
    if ecls is not None:
        for _ in range(120):
            engine.step_model()

        row.update({
            "ecls_running": bool(getattr(ecls, "ecls_running", False)),
            "ecls_blood_flow": round(float(getattr(ecls, "blood_flow", 0.0) or 0.0), 6),
            "ecls_p_ven": round(float(getattr(ecls, "p_ven", 0.0) or 0.0), 3),
            "ecls_p_art": round(float(getattr(ecls, "p_art", 0.0) or 0.0), 3),
            "ecls_p_tmp": round(float(getattr(ecls, "p_tmp", 0.0) or 0.0), 3),
        })

    vent = engine.models.get("VENT")
    if vent is not None:
        for _ in range(120):
            engine.step_model()

        row.update({
            "vent_mode": getattr(vent, "vent_mode", ""),
            "vent_pres": round(float(getattr(vent, "pres", 0.0) or 0.0), 3),
            "vent_flow": round(float(getattr(vent, "flow", 0.0) or 0.0), 3),
            "vent_minute_volume": round(float(getattr(vent, "minute_volume", 0.0) or 0.0), 6),
        })

    resus = engine.models.get("RESUS")
    if resus is not None:
        if hasattr(resus, "switch_cpr"):
            resus.switch_cpr(True)
        for _ in range(300):
            engine.step_model()

        row.update({
            "resus_cpr_enabled": bool(getattr(resus, "cpr_enabled", False)),
            "resus_chest_comp_pres": round(float(getattr(resus, "chest_comp_pres", 0.0) or 0.0), 3),
            "resus_chest_comp_no": int(getattr(resus, "chest_comp_no", 0) or 0),
        })

    monitor = engine.models.get("MON")
    if monitor is not None:
        for _ in range(120):
            engine.step_model()

        row.update({
            "mon_hr": round(float(getattr(monitor, "heart_rate", 0.0) or 0.0), 3),
            "mon_abp_signal": round(float(getattr(monitor, "abp_signal", 0.0) or 0.0), 3),
            "mon_etco2": round(float(getattr(monitor, "etco2", 0.0) or 0.0), 3),
            "mon_spo2_pre": round(float(getattr(monitor, "spo2_pre", 0.0) or 0.0), 3),
        })

    return row

snapshot_results = [
    run_snapshot("baseline.json"),
    run_snapshot("shunts.json"),
    run_snapshot("placenta.json"),
    run_snapshot("fluids.json"),
    run_snapshot("ans.json"),
    run_snapshot("ecls.json"),
    run_snapshot("ventilator.json"),
    run_snapshot("resuscitation.json"),
    run_snapshot("monitor.json"),
]

if "results" in globals():
    del results

comparison_results = snapshot_results
snapshot_results

[{'definition': 'baseline.json',
  'has_shunts': True,
  'has_placenta': False,
  'has_fluids': False,
  'has_ans': False,
  'has_ecls': False,
  'has_ventilator': False,
  'has_resuscitation': False,
  'has_monitor': False,
  'res_fo': 343.803,
  'res_vsd': 573.005,
  'velocity_fo': -6.3662,
  'velocity_vsd': 12.7324},
 {'definition': 'shunts.json',
  'has_shunts': True,
  'has_placenta': False,
  'has_fluids': False,
  'has_ans': False,
  'has_ecls': False,
  'has_ventilator': False,
  'has_resuscitation': False,
  'has_monitor': False,
  'res_fo': 343.803,
  'res_vsd': 573.005,
  'velocity_fo': -6.3662,
  'velocity_vsd': 12.7324},
 {'definition': 'placenta.json',
  'has_shunts': False,
  'has_placenta': True,
  'has_fluids': False,
  'has_ans': False,
  'has_ecls': False,
  'has_ventilator': False,
  'has_resuscitation': False,
  'has_monitor': False,
  'umb_art_flow': 0.003333,
  'umb_ven_flow': -0.36,
  'placenta_running': True,
  'umb_clamped': False},
 {'definition': 'fluids.jso